In [1]:
# |default_exp daft_embeddings

In [2]:
# | hide
%load_ext autoreload
%autoreload 2

In [1]:
#| export

import daft
import timm
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from einops import rearrange
from dataclasses import dataclass, field
from typing import Callable

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#| export

daft.set_execution_config(enable_native_executor=True,
                          default_morsel_size=50)

DaftContext(_daft_execution_config=<daft.daft.PyDaftExecutionConfig object at 0x7faca4a81130>, _daft_planning_config=<daft.daft.PyDaftPlanningConfig object at 0x7faca4a81410>, _runner=None)

In [4]:
#| hide

tmp_path = Path("./tmp-test-images")

# img2dataset --url_list https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv --output_folder ./images --processes_count 10 --thread_count 5 --number_sample 2000

from subprocess import run

cmd = ["img2dataset","--url_list",
       "https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv",
       "--output_folder", tmp_path.as_posix(),\
       "--processes_count", "10",
        "--thread_count", "5",
        "--number_sample", "2000"]

run(cmd)

Starting the downloading of this file
Sharding file number 1 of 1 called https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv


0it [00:00, ?it/s]

File sharded in 175 shards


26it [00:47,  1.01it/s]

worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7187 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7187 - count: 10000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7420 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 14332 - count: 20000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7299 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 21421 - count: 30000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7785 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 14658 - count: 40000
worker  - success: 0.000 - failed to download: 1.000 - failed

...

worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7316 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 34083 - count: 1210000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7622 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 33094 - count: 1220000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 6978 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 33365 - count: 1230000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 7430 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 33636 - count: 1240000
worker  - success: 0.000 - failed to download: 1.000

175it [01:31,  1.91it/s]


worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 9452 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 31376 - count: 1510000
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 8256 - count: 3043
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 31439 - count: 1513043
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 9768 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 31584 - count: 1523043
worker  - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 6890 - count: 10000
total   - success: 0.000 - failed to download: 1.000 - failed to resize: 0.000 - images per sec: 31792 - count: 1533043
worker  - success: 0.000 - failed to download: 1.000 

ERROR: Could not consume arg: --number_sample
Usage: img2dataset --url_list https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv --output_folder tmp-test-images --processes_count 10 --thread_count 5 -

For detailed information on this command, run:
  img2dataset --url_list https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv --output_folder tmp-test-images --processes_count 10 --thread_count 5 - --help


CompletedProcess(args=['img2dataset', '--url_list', 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable.csv', '--output_folder', 'tmp-test-images', '--processes_count', '10', '--thread_count', '5', '--number_sample', '2000'], returncode=2)

In [ ]:
#| export

@dataclass
class TimmEmbedder:
    """
    embed an image with any timm model that supports this

    Reference: https://huggingface.co/docs/timm/main/en/feature_extraction#pooled
    """
    model_name: str
    device: torch.device = field(init=False)
    dtype: torch.dtype = field(init=False)
    _model: Callable = field(init=False)
    _transforms: Callable = field(init=False)


    def __post_init__(self):
        # initialize model and transforms
        self._model = timm.create_model(self.model_name, pretrained=True, num_classes=0)
        cfg = self._model.pretrained_cfg
        self._transform = timm.data.create_transform(**timm.data.resolve_data_config(cfg))

        # set device and dtype
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dtype = torch.bfloat16 if self.device.type == "cuda" and torch.cuda.is_bf16_supported() else (
            torch.float16 if self.device.type == "cuda" else torch.float32
        )
        print(f"Inference device: {self.device} with dtype: {self.dtype}")

        # optimize model for inference
        self._model = self._model.eval().to(device=self.device, dtype=self.dtype)
        self._model = torch.jit.optimize_for_inference(torch.jit.script(self._model))


    def __call__(self, image: torch.Tensor | Image.Image) -> np.array:
            """transform image, run inference, extract embedding as 1D array"""
            image = self._transform(image).to(self.device, self.dtype).unsqueeze(0)
            emb = self._model(image)
            return emb.detach().cpu().float().numpy().squeeze()


In [6]:
#| hide

# it's a one-stop shop
# e = TimmEmbedder("mobilenetv3_large_100")
# e(torch.rand((3,256,256)))

In [7]:
#| export

@daft.udf(return_dtype=daft.DataType.list(daft.DataType.float32()))
class EmbedImages:
    """run timm embedder on an image column"""
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.embedder = TimmEmbedder(self.model_name)

    def __call__(self, images_col):
        images = [rearrange(im, "h w c -> c h w") for im in images_col.to_pylist()]
        ## Note: expect images are different sizes
        ## could maybe speed up by doing the resize transform separately,
        ## then doing batch inference
        ## Example: https://colab.research.google.com/github/Eventual-Inc/Daft/blob/main/tutorials/mnist.ipynb
        return [self.embedder(
            torch.tensor(im, dtype=self.embedder.dtype)) for im in images]

## Test it out

In [ ]:
#| hide

glob = tmp_path.as_posix() +"/*.jpg"
images_df = daft.from_glob_path(glob).with_column_renamed("path", "path_full_img")

images_df = images_df.with_column("image", daft.col("path_full_img"
                                 ).url.download().image.decode(
                                     mode="RGB", on_error="null"))

/home/willsa/git/bedmap-dev/.venv/lib/python3.12/site-packages/daft/context.py:168: UserWarning: Daft is configured to use the new NativeRunner by default as of v0.4.0. If you are encountering any regressions, please switch back to the legacy PyRunner via `daft.context.set_runner_py()` or by setting the env variable `DAFT_RUNNER=py`. We appreciate you filing issues and helping make the NativeRunner better: https://github.com/Eventual-Inc/Daft/issues
  warnings.warn(


## time some embedding models on CPU on my laptop

- "mobilenetv3_large_100" : 3m0s
- "vit_large_patch14_reg4_dinov2.lvd142m" : OOM
- "vit_base_patch14_reg4_dinov2.lvd142m" :

In [ ]:
#| hide

MODEL_NAME = "vit_base_patch14_reg4_dinov2.lvd142m"
EmbedImagesWithModel = EmbedImages.with_init_args(MODEL_NAME)

images_df = images_df.with_column("embed", EmbedImagesWithModel(daft.col("image")))

: 

In [ ]:
#| hide

images_df = images_df.collect()

🗡️ 🐟 Project: 00:00 1,650 rows received, 0 rows emittedInference device: cpu with dtype: torch.float32Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32

Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32
Inference device: cpu with dtype: torch.float32


In [ ]:
#| hide

tmp_path.unlink()